## Cluster Mouse Samples

**Samples:** 
- RS-TMA-5

**Method**: Cluster Umap with Leiden algorithm 

Compare to images

compare to histological subtypes

In [ ]:
#load libraries
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import scipy
import scanpy as sc
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
mpl.rc('figure', max_open_warning = 0)
codedir = os.getcwd()
import phenograph
os.chdir('/home/groups/graylab_share/OMERO.rdsStore/engje/Data')
from mplex_image import visualize as viz, process, preprocess, normalize, mics, mpimage
np.random.seed(1027)

In [ ]:
codedir = '/home/groups/graylab_share/OMERO.rdsStore/engje/Data/20210000/cmIF_2021-07-07_RS-mTMA'

In [ ]:
#change to correct directory
datadir =  f'{codedir}'
s_date = '20211110'#'20211029'
os.chdir(codedir)
if not os.path.exists(s_date):
    os.mkdir(s_date)
%matplotlib inline

# Table of contents <a name="contents"></a>
1. [Load manual](#loadold)
2. [Load intensity](#load)
4. [Normalize](#leiden)
5. [Umap](#l7)
6. [Leiden](#l8)
[annotate](#lbar)
5. [Sub-cluster](#kmeans)
8. [Visualize](#viz)

# Load Gated Celltypes <a name="loadold"></a>

[contents](#contents)

In [ ]:
s_out = '20211109_RS-mTMA-5'
df_man = pd.read_csv(f'{s_out}_GatedPositiveCellNames.csv',index_col=0)

In [ ]:
#df_man.index = df_man.jindex
#df_man.index = [item.replace('ad60','2021.02.03-adeno60').replace('ad82','2021.02.04-adeno82') for item in df_man.index]

# Load Mean Intensity <a name="load"></a>

load data

[contents](#contents)

In [ ]:
#sample
ls_sample = ['RS-mTMA-5']

In [ ]:
#mean intensity and centroid XY
df_xy = pd.DataFrame()
df_mi = pd.DataFrame()
for s_sample in ls_sample:
    #1027
    #df_mi = df_mi.append(pd.read_csv(f'{datadir}/features_{s_sample}_patched_MeanIntensity_Shape_DAPI1_DAPI8_subtractedregisteredimages.csv',index_col=0))
    #1029
    df_mi = df_mi.append(pd.read_csv(f'{datadir}/features_{s_sample}_patched_MeanIntensity_Shape__subtractedregisteredimages.csv',index_col=0))
    df_xy = df_xy.append(pd.read_csv(f'{datadir}/features_{s_sample}_CentroidXY.csv',index_col=0))


In [ ]:
sorted(df_mi.columns)

In [ ]:
#df_mi.columns = [item.split('_')[0] for item in df_mi.columns]
ls_drop = ['DAPI0_nuclei', 'DAPI1_nuclei', 'DAPI2_nuclei', 'DAPI3_nuclei', 'DAPI4_nuclei', 'DAPI5Q_nuclei', 'DAPI5_nuclei',
 'DAPI6_nuclei', 'DAPI7_nuclei', #'DAPI8_nuclei','R1c2_nuclei',(AF)
           'R0c2_nuclei', 'R0c2_perinuc5', 'R0c3_nuclei', 'R0c3_perinuc5', 'R0c4_nuclei',
 'R0c4_perinuc5', 'R0c5_nuclei', 'R0c5_perinuc5',  'R1c2_perinuc5', 'R4c2_nuclei', 'R4c2_perinuc5', 'R5Qc2_nuclei',
 'R5Qc2_perinuc5', 'R5Qc3_nuclei', 'R5Qc3_perinuc5', 'R5Qc4_nuclei', 'R5Qc4_perinuc5', 'R5Qc5_nuclei', 'R5Qc5_perinuc5','cell_area',
 'cell_eccentricity', 'cell_euler', 'cytoplasm_area', 'cytoplasm_eccentricity', 'cytoplasm_euler','Ecad_negative',
 'EpCAM_cellmem2p25','Ecad_cellmem2p25',
 #failed
 'CC3-488_nuclei', 'CC3-647_nuclei', 'CD4_perinuc5','CD3_perinuc5','F4-80_perinuc5','MHCII_cytoplasm','ELN_perinuc5','NFkB_cytoplasm',
 'Arg1_cytoplasm','panCK_cytoplasm',
          ]
# Failed':{'CC3-647','CD4','CD3','F4-80','MHCII','ELN','NFkB','CC3-488', 'panCK','Arg1',}
df_mi = df_mi.loc[:,~df_mi.columns.isin(ls_drop)]
df_mi.rename({'nuclei_area':'area','nuclei_eccentricity':'eccentricity','slide_scene':'slidescene'},axis=1,inplace=True)
#df_xy.rename({'DAPI_X':'DAPI_X_old','DAPI_Y':'DAPI_Y_old','slide_scene':'slide_scene_old'},axis=1,inplace=True)

In [ ]:
df_mi.columns = [item.split('_')[0] for item in df_mi.columns]
df_mi.columns

In [ ]:
df_exclude = pd.DataFrame()
for s_file in os.listdir():
    if s_file.find('exclude_RS') > -1:
        print(s_file)
        df_exclude = df_exclude.append(pd.read_csv(s_file,index_col=0))

In [ ]:
#df_mi[(~df_mi.index.isin(df_exclude.index))]
#df_mi[(df_mi.DAPI8 > 1000)]

In [ ]:
df_filter_mi = df_mi[(df_mi.DAPI8 > 1000) & (~df_mi.index.isin(df_exclude.index))].merge(df_xy.loc[:,['DAPI_X', 'DAPI_Y']], left_index=True,right_index=True)
df_filter_mi.rename({'slidescene':'slide_scene'},axis=1,inplace=True)

In [ ]:
#df_filter_mi#.head()
#s_date

In [ ]:
s_out = f'{codedir}/{s_date}_{s_sample}_FilteredMeanIntensity.csv' #1027 old, 1029 updated
if not os.path.exists(s_out):
    print('saving csv')
    df_filter_mi.to_csv(s_out)

# Normalize <a name="leiden"></a>

background subtraction


[contents](#contents)

In [ ]:
os.chdir(codedir)
s_date = '20211027' #326562 rows
s_date = '20211029' #748728  rows
s_date = '20211109' #722760 rows
s_date = '20211110' #718696 rows 16 neighbors
s_sample = f'{s_date}_RS-mTMA-5'
df_mi = pd.read_csv(f'{s_sample}_FilteredMeanIntensity.csv',index_col=0)


In [ ]:
'''
b_control = ((df_both.index.str.contains('JE-TMA')) & (df_both.slide_scene!='JE-TMA-42_scene12')) # & (df_mi.loc[:,ls_pos].isna().sum(axis=1)==0)
#fit
data = df_both.loc[b_control,ls_pos].T
batch = df_both.loc[b_control,'batch']
gamma_star, delta_star, stand_mean, var_pooled = normalize.combat_fit(data, batch)
#transform
#data = df_both.loc[:,ls_pos].T
#batch = df_both.loc[:,'batch']
#bayesdata = normalize.combat_transform(data,batch,gamma_star,delta_star,stand_mean, var_pooled)
#df_norm = bayesdata.T
'''

In [ ]:
#subtract background

d_bg = {'CD31':8,#11,
        'CD45':3, 'CD8':10, 'ColIV':12, 'FoxP3':18,#12,
 'CD103':10, 'CD11b':1.5, 'CD11c':6
}
df_bg = df_mi.copy()
for s_marker, i_thresh in d_bg.items():
    df_bg[s_marker] = np.clip(df_mi.loc[:,s_marker] - i_thresh*256, a_min=0, a_max=None)

s_out = f'{codedir}/{s_sample}_BGSubtractedMeanIntensity.csv'
if not os.path.exists(s_out):
    print('saving csv')
    df_bg.to_csv(s_out)

## Umap <a name="l7"></a>


[contents](#contents)

In [ ]:
#load annotation
import re
df_a = pd.read_csv('RS-MYC-PTENfl_TMA_subgroups.csv')
df_a.loc[~df_a.GroupA.isna(),'Groups'] = 'stroma_rich'
df_a.loc[~df_a.GroupB.isna(),'Groups'] = 'stroma_poor'
df_a.loc[df_a.Groups.isna(),'Groups'] = 'not_tumor'
df_a['Xiaoyan'] = df_a.Xiaoyan.fillna(df_a.TMA)
df_a.index = df_a.TMA #drop(['oldE1','oldF6'],inplace=True)
d_a = dict(zip(df_a.index,df_a.Groups))
df_a['slide_scene'] = [f'RS-mTMA-5_scene{re.sub(r"[0-9]+", "", item)}{int(re.sub(r"[A-Z]+", "", item)):02d}' for item in df_a.index]
df_a['Group'] = df_a.Xiaoyan.map(d_a).fillna('not_tumor')      

In [ ]:
#df_a.loc['A1']
#df_a.loc['B1']
#df_a.loc['C1']
#df_a.loc['D1']
#df_a.loc['E1'] 20211110
#df_a.loc['G6']

In [ ]:
s_sample = '20211110_RS-mTMA-5'
b_scale=True
s_date = s_sample.split('_')[0]
#22 features cd11b monocytes, granulocytes, macrophages, and natural killer cells
#cd11c high levels on most human dendritic cells, but also on monocytes, macrophages, neutrophils, and some B cells 
lls_col = [[ 'Ki67', 'LamAC', 'SOX9',#'FoxP3',
       'pMYC', 'CD103', 'CD11b', 'CD11c', 'CD31', 'CD45', 'CD8', 'ColIV',
       'ColVI', 'CK5', 'Ecad', 'EpCAM', 'Gal3', 'S100A6', 'Vim', 'aSMA',
       'area', 'eccentricity','DAPI8','R1c2'],
     [ 'Ki67', 'LamAC', 'SOX9', #'FoxP3','CD103','CD11c', 'CD31','CD8',
       'pMYC',  'CD11b',  'CD45',  'ColIV',
       'ColVI', 'CK5', 'Ecad', 'EpCAM', 'Gal3', 'S100A6', 'Vim', 'aSMA',
       'area', 'eccentricity']]

for s_type in ['BGSubtracted','Filtered']:
    df_mi = pd.read_csv(f'{s_sample}_{s_type}MeanIntensity.csv',index_col=0)
    for ls_col in lls_col:
        print(len(ls_col))
        break
        adata = sc.AnnData(df_mi.loc[:,ls_col])
        #raw
        adata.raw = adata
        #reduce dimensionality
        sc.tl.pca(adata, svd_solver='auto')
        fig,ax=plt.subplots(figsize=(3.5,5))
        sc.pl.highest_expr_genes(adata, n_top=48,ax=ax,save=f'Expression_{len(ls_col)}_{s_type}.png')
        plt.tight_layout()
        sc.pl.pca_variance_ratio(adata,n_pcs=32, log=True)
        adata.obs['slide'] = [item.split('_cell')[0] for item in adata.obs.index]
        if b_scale:
            sc.pp.scale(adata, zero_center=False, max_value=20)
            s_sample = s_sample + '_s'
        #load annotation
        # mapping
        d_a = dict(zip(df_a.slide_scene,df_a.Group))
        adata.obs['Group'] = adata.obs.slide.map(d_a)

        #umap
        for n_neighbors in [15]: #,30
            results_file = f'{s_date}/{s_sample}_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.h5ad'
            if os.path.exists(results_file):
                print('loading umap')
                adata = sc.read_h5ad(results_file)
                # mapping
                d_a = dict(zip(df_a.slide_scene,df_a.Group))
                adata.obs['Group'] = adata.obs.slide.map(d_a)
            else:
                print('calc umap')
                # calculate neighbors 
                sc.pp.neighbors(adata, n_neighbors=n_neighbors) #, method='rapids'
                sc.tl.umap(adata)
            #save results
            if not os.path.exists(results_file):
                adata.write(results_file)
            #color by markers
            
            figname = f"Umap_markers_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
            title=figname.split('.png')[0].replace('_',' ')
            sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p99.5',ncols=6,save=figname)
            #color by slide
            figname = f"Umap_slide_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
            title=figname.split('.png')[0].replace('_',' ')
            fig,ax = plt.subplots(figsize=(3,3), dpi=200)
            sc.pl.umap(adata, color='slide',wspace=.25,save=figname,ax=ax,title=title)
            
            #color by group
            figname = f"Umap_Group_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
            title=figname.split('.png')[0].replace('_',' ')
            fig,ax = plt.subplots(figsize=(3,3), dpi=200)
            sc.pl.umap(adata, color='Group',wspace=.25,save=figname,ax=ax,title=title)
            break
        break
    break


In [ ]:
#screen all
for s_slide in adata.obs.slide.unique():
    adata.obs['test'] = (adata.obs.slide == s_slide).replace({True:1,False:0})
    sc.pl.umap(adata, color='test',title=s_slide)
    break

In [ ]:
# weird ones (fixed)
s_slide = 'RS-mTMA-5_sceneE03' #corner extra
s_slide = 'RS-mTMA-5_sceneH11' # circel imaging artifact (pmyc)
s_slide = 'RS-mTMA-5_sceneI02' #cd31 bg
#Still unique
s_slide = 'RS-mTMA-5_sceneA05' #high foxp3 bg, some float
s_slide = 'RS-mTMA-5_sceneE07' #folded tissue
s_slide = 'RS-mTMA-5_sceneH06' #bright floating tissue
s_slide = 'RS-mTMA-5_sceneI01' #bright floating tissue
s_slide = 'RS-mTMA-5_sceneI11' #epcam, ecad, CK5 neg (no tumor)
s_slide = 'RS-mTMA-5_sceneI07' #dried out bottom? pmyc weird
#new ouliers
#A05, E06, E07, F03, G09, H06, I01, I07, I11
#other tissues
# A11 spleen B10 spleen?
#B6 lung #I10 lung
#d4 normal pad, d9 adj #d12 fat pad
# E6, E12 lymph node I12 lymph node
#for s_slide in adata.obs.slide.unique():
adata.obs['test'] = (adata.obs.slide == s_slide).replace({True:1,False:0})
sc.pl.umap(adata, color='test',title=s_slide)

## Cluster Leiden <a name="l8"></a>

main cell types


[contents](#contents)

In [ ]:
len(ls_col)

In [ ]:
s_sample = '20211110_RS-mTMA-5_s'
s_type = 'BGSubtracted'
s_date = s_sample.split('_')[0]
n_neighbors = 15
n_markers = 23 #24

results_file = f'{s_date}/{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}.h5ad'
adata = sc.read_h5ad(results_file)

In [ ]:
#leiden
for resolution in [0.4, 0.5,0.6]: #0.5,
    results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
    if not os.path.exists(results_file):
        sc.tl.leiden(adata,resolution=resolution)
        adata.write(results_file)
    else:
        print('loading')
        adata = sc.read_h5ad(results_file)
    fig,ax = plt.subplots(figsize=(3,3),dpi=200)
    figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}.png'
    sc.pl.umap(adata, color='leiden',ax=ax,title=figname.split('.png')[0].replace('_',' '),wspace=.25,save=figname,)
    #fig.savefig(f'./figures/{figname}')
    #break

## Annotate Leiden <a name="lbar"></a>

main cell types


[contents](#contents)

In [ ]:
s_sample ='20211110_RS-mTMA-5_s'
s_type = 'BGSubtracted'
s_date = s_sample.split('_')[0]
n_neighbors = 15#16
n_markers = 24
resolution = 0.6
results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
adata = sc.read_h5ad(results_file)

In [ ]:
results_file

In [ ]:
df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
df_p['leiden'] = adata.obs['leiden']
#CD45 above 300
#CK5 above 2048 (768)
#ecad/epcam above 2000 (768)
# CD68 above 1536
#area above 318
#all 30 neigh, 7 pcs
b_annot = False
if results_file=='20211110_RS-mTMA-5_s_15neighbors_24markers_BGSubtracted_leiden0.6.h5ad':
        tum_clust = [] 
        imm_clust = []
        str_clust = []
        endo_clust = []
    
elif resolution == 0.4:  
    if n_neighbors == 15:
        tum_clust = ['3', '5', '10', '0', '14', '11', '6', '17',   '13'] 
        imm_clust = ['14']
        str_clust = ['1', '16', '2', '4', '9','12','8']
        endo_clust = ['15','7']
elif resolution == 0.5:
    if n_neighbors == 16:
        tum_clust = ['7', '1',  '5', '8', '2', '11', '16','12'] #15
        str_clust = ['0', '10',   '3','16','14','6','9','13','4','15','18'] #'12',
        imm_clust = ['16','14','6'] # []
        endo_clust = ['17','9'] # []
    elif n_neighbors == 15:
        tum_clust = ['1', '4', '0', '15', '11', '19', '9', '5', '22', '18','13','6','21']
        str_clust = ['10', '12', '16', '17', '2', '3',  '7', '8']
        imm_clust = ['19'] 
        endo_clust = ['14','20']
else:
    print('not annot')
    b_annot = False

df_p.groupby('leiden').mean().CD45.sort_values(ascending=False)#.index
#set(df_p.groupby('leiden').mean().area.sort_values(ascending=False).index) - set(tum_clust + imm_clust + endo_clust)
#set(tum_clust)

In [ ]:
#clustermap
#df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
#df_p['leiden'] = adata.obs['leiden']
d_replace = {}
d_replace.update(dict(zip(tum_clust,[f'{item}\nep' for item in tum_clust])))
d_replace.update(dict(zip(str_clust,[f'{item}\ns' for item in str_clust])))
d_replace.update(dict(zip(imm_clust,[f'{item}\nim' for item in imm_clust])))
d_replace.update(dict(zip(endo_clust,[f'{item}\nen' for item in endo_clust])))
if b_annot:
    df_plot = df_p.groupby('leiden').mean().rename(d_replace)
else:
    df_plot = df_p.groupby('leiden').mean()
df_plot.index.name = f'leiden {resolution}'
g = sns.clustermap(df_plot,z_score=1,figsize=(8,8),cmap='viridis',
                   vmin=-2,vmax=2,method='ward')
#g.fig.suptitle(f'leiden {resolution}',x=.9) 
g.savefig(f'{codedir}/{s_date}/{s_sample}_clustermap_leiden_{resolution}.png',dpi=200)
marker_genes = df_p.groupby('leiden').mean().iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_p.groupby('leiden').mean().iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

In [ ]:
ls_merge = ['20']
#ls_merge=str_clust
adata.obs['test'] = (adata.obs.leiden.isin(ls_merge)).replace({True:1,False:0})
sc.pl.umap(adata, color='test',title='merged')

In [ ]:
ls_merge = ['6']
#ls_merge=str_clust
adata.obs['test'] = (adata.obs.leiden.isin(ls_merge)).replace({True:1,False:0})
sc.pl.umap(adata, color='test',title='merged')

In [ ]:
#plot all groups spatially - leiden 
from mplex_image import analyze
df_data = df_man
colors = mpl.cm.tab20b.colors + mpl.cm.tab20c.colors

df_pos = analyze.celltype_to_bool(df_p,'leiden')
df_pos.columns = [str(item) for item in df_pos.columns]


In [ ]:
s_clust = '17'
s_slide = 'RS-mTMA-5_sceneB06'#'RS-mTMA-5_sceneC03'#'RS-mTMA-5_sceneH03'#'RS-mTMA-5_sceneI10'#'cene021'
fig,ax = plt.subplots(figsize=(5,4.5),dpi=200)
#plot negative cells
df_scene = df_data[df_data.index.str.contains(s_slide)]
ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=0.1,label=f'')
for idxs, s_color in enumerate(df_pos.columns):
    s_color = s_clust
    s_color = str(s_color)
    if len(df_p[(df_p.index.str.contains(s_slide)) & (df_pos.loc[:,s_color])])>=1:
        #plot positive cells
        ls_index = df_p[(df_p.index.str.contains(s_slide)) & (df_pos.loc[:,s_color])].index
        ax.scatter(data=df_data.loc[ls_index],x='DAPI_X',y='DAPI_Y',label=f'{s_color}',s=0.1,color=colors[idxs])
    break
ax.set_title(f"{s_slide}", fontsize=16) # \n {d_a[s_slide]}
ax.axis('equal')
ax.set_ylim(ax.get_ylim()[::-1])
plt.legend(markerscale=10,framealpha=.5) 


In [ ]:
fig,ax=plt.subplots(figsize=(2.5,5),dpi=200)
df_p.groupby('leiden').count().loc[categories_order[::-1]].iloc[:,1].plot(kind='barh',title='Cell Count',ax=ax)
fig.savefig(f'./{s_date}/barplot_{n_neighbors}neighbors_{len(marker_genes)}markers_leiden{resolution}.png')

In [ ]:
df_p.loc[df_p.leiden.isin(tum_clust),'leiden_celltype'] = 'epithelial'
df_p.loc[df_p.leiden.isin(str_clust),'leiden_celltype'] = 'stromal'
df_p.loc[df_p.leiden.isin(imm_clust),'leiden_celltype'] = 'immune'
df_p.loc[df_p.leiden.isin(endo_clust),'leiden_celltype'] = 'endothelial'
#df_p.leiden_celltype = df_p.leiden_celltype.fillna('NOS')

In [ ]:
adata.obs['leiden_celltype'] = df_p.leiden_celltype
fig,ax = plt.subplots(figsize=(3,3),dpi=200)
figname=f'leiden__{resolution}_leiden_celltype.png'
sc.pl.umap(adata, color='leiden_celltype',ax=ax,title=figname.split('.png')[0].replace('_',' '),save=figname,wspace=0.25) #


## accuracy

In [ ]:
#df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)

In [ ]:
#merge
df_p = df_p.merge(df_man.loc[:,['celltype']],left_index=True,right_index=True,how='left')

In [ ]:
df_p.loc[df_p.celltype=='tumor','celltype'] = 'epithelial'

In [ ]:
import sklearn
#df_p['cell_type'] = df_p.celltype.fillna('NOS')
labels = sorted(df_p.leiden_celltype.unique())
confusion_matrix = sklearn.metrics.confusion_matrix(df_p.celltype,df_p.leiden_celltype,labels=labels)
#accuracy
i_acc = sklearn.metrics.accuracy_score(df_p.celltype,df_p.leiden_celltype)
disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix,display_labels=labels)
fig, ax = plt.subplots(figsize = (5,5))
disp.plot(xticks_rotation='vertical',ax=ax) 
ax.set_title(f'Leiden {resolution} \n {i_acc}')
plt.tight_layout()
fig.savefig(f'{s_date}/Confusion_matrix_leiden{resolution}.png',dpi=200)

## manual

umap

In [ ]:
adata.obs['manual_celltype'] = df_p.celltype
fig,ax = plt.subplots(figsize=(3,3),dpi=200)
figname=f'{s_sample}_celltype_manual.png'
sc.pl.umap(adata, color='manual_celltype',ax=ax,save=figname,wspace=0.25,title=figname.split('.png')[0].replace('_',' ')) #

In [ ]:
s_date

# gate stromal

In [ ]:
df_p['gated_celltype'] = df_p.leiden_celltype
#gated cells
df_p.loc[((df_p.leiden_celltype == 'stromal') & (df_p.CD31 > 0)),'gated_celltype'] = 'endothelial'
df_p.loc[((df_p.leiden_celltype == 'stromal') & (df_p.CD45 > 0) & (df_p.CD31 <= 0)),'gated_celltype'] = 'immune'

In [ ]:
adata.obs['gated_celltype'] = df_p.gated_celltype
fig,ax = plt.subplots(figsize=(3,3),dpi=200)
figname=f'_gated_celltype.png'
sc.pl.umap(adata, color='gated_celltype',ax=ax,save=figname,title=figname.split('.png')[0].replace('_',' ')) #

In [ ]:
s_sample
resolution
n_neighbors

In [ ]:
df_p.merge(df_man.loc[:,['DAPI_X','DAPI_Y']],left_index=True,right_index=True).to_csv(f'results_{s_sample}_leiden{resolution}_gated_celltype.csv')

# Sub Cluster  <a name="kmeans"></a>

re-cluster immune and fibroblast

doesn't work great: gating better
 

[contents](#contents)

In [ ]:
s_sample = '20211110_RS-mTMA-5'
resolution = 0.5
df = pd.read_csv(f'results_{s_sample}_leiden{resolution}_gated_celltype.csv',index_col=0)
# cluster 5 8 and 0 are immune
s_markers = 'stromal'
print((df.leiden_celltype=='stromal').sum()/len(df))
ls_marker = ['FoxP3', 'Ki67', 'CD103', 'CD11b', 'CD11c',
       'CD31', 'CD45', 'CD8', 'ColIV', 'ColVI',  'Gal3',
       'S100A6', 'Vim', 'aSMA', 'area', 'eccentricity', 'DAPI8', 'R1c2']
adata = sc.AnnData(df.loc[df.leiden_celltype=='stromal',ls_marker])
adata.raw = adata

In [ ]:
#adata.var.index
#leiden
for resolution in [0.6]: #0.4, 0.5, 0.6
    results_file = f'{s_sample}_{s_markers}_markers_leiden{resolution}.h5ad'
    print(results_file)
    if not os.path.exists(results_file):
        print('umap')
        #reduce dimensionality
        sc.tl.pca(adata, svd_solver='auto')
        sc.pp.neighbors(adata, n_neighbors=15)
        sc.tl.umap(adata)
        sc.tl.leiden(adata,resolution=resolution)
        adata.write(results_file)
    else:
        adata = sc.read_h5ad(results_file)
    fig,ax = plt.subplots(figsize=(3,3),dpi=200)
    figname=f'leiden__{s_markers}_{resolution}.png'
    sc.pl.umap(adata, color='leiden',ax=ax,title=figname.split('.png')[0].replace('_',' ')) #save=figname,
    #color by markers

    figname = f"Umap_subcluster_markers_{s_markers}_markers.png"
    title=figname.split('.png')[0].replace('_',' ')
    sc.pl.umap(adata, color=ls_marker,vmin='p1.5',vmax='p99.5',ncols=6,save=figname)
    break

In [ ]:
#clustermap
df_p = pd.DataFrame(data=adata.X,index=adata.obs.index,columns=adata.var.index)
df_p['leiden'] = adata.obs['leiden']
#d_replace = {}
#d_replace.update(dict(zip(tum_clust,[f'{item}\nep' for item in tum_clust])))
#d_replace.update(dict(zip(imm_clust,[f'{item}\nim' for item in imm_clust])))
#d_replace.update(dict(zip(str_clust,[f'{item}\ns' for item in str_clust])))
#d_replace.update(dict(zip(endo_clust,[f'{item}\nen' for item in endo_clust])))
df_plot = df_p.groupby('leiden').mean().rename(d_replace)
df_plot.index.name = f'leiden {resolution}'
g = sns.clustermap(df_plot,z_score=1,figsize=(8,8),cmap='viridis',
                   vmin=-2,vmax=2)
#g.fig.suptitle(f'leiden {resolution}',x=.9) 
g.savefig(f'{codedir}/20210316_figures/{s_sample}_clustermap_leiden_{resolution}_{n_markers}.png',dpi=200)
marker_genes = df_p.groupby('leiden').mean().iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_p.groupby('leiden').mean().iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

In [ ]:
#cluster 2 and 3 are FB
print(df.leiden.isin([2,3]).sum()/len(df))

# Visualize  <a name="viz"></a>
 

[contents](#contents)

In [ ]:
s_sample = '20211110_RS-mTMA-5'
resolution = 0.5
s_date = s_sample.split('_')[0]

In [ ]:
#load annotation
import re
df_a = pd.read_csv('RS-MYC-PTENfl_TMA_subgroups.csv')
df_a.loc[~df_a.GroupA.isna(),'Groups'] = 'stroma_rich'
df_a.loc[~df_a.GroupB.isna(),'Groups'] = 'stroma_poor'
df_a.loc[df_a.Groups.isna(),'Groups'] = 'not_tumor'
df_a['Xiaoyan'] = df_a.Xiaoyan.fillna(df_a.TMA)
df_a.index = df_a.TMA #drop(['oldE1','oldF6'],inplace=True)
d_a = dict(zip(df_a.index,df_a.Groups))
df_a['slide_scene'] = [f'RS-mTMA-5_scene{re.sub(r"[0-9]+", "", item)}{int(re.sub(r"[A-Z]+", "", item)):02d}' for item in df_a.index]
df_a['Group'] = df_a.Xiaoyan.map(d_a).fillna('not_tumor')      



In [ ]:
from mplex_image import analyze
df_p = pd.read_csv(f'results_{s_sample}_leiden{resolution}_gated_celltype.csv',index_col=0)
df_p['slide_scene'] = [item.split('_cell')[0] for item in df_p.index]
df_p['slide'] = [item.split('_')[0] for item in df_p.index]
#annotate
d_a = dict(zip(df_a.slide_scene,df_a.Group))
df_p['Group'] = df_p.slide_scene.map(d_a)

In [ ]:
#plot all groups spatially - leiden 
df_pos = analyze.celltype_to_bool(df_p,'leiden')
df_pos.columns = [str(item) for item in df_pos.columns]
for s_slide in sorted(set(df_p.slide_scene)):
    s_slide = 'RS-mTMA-5_sceneB07'
    s_slide = 'RS-mTMA-5_sceneA09'
    fig,ax = plt.subplots(figsize=(5,4.5),dpi=200)
    #plot negative cells
    df_scene = df_p[df_p.index.str.contains(s_slide)]
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=0.1,label=f'')
    #for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
    for idxs, s_color in enumerate(df_pos.columns):
        s_color = str(s_color)
        if len(df_p[(df_p.slide_scene==s_slide) & (df_pos.loc[:,s_color])])>=1:
            #plot positive cells
            ax.scatter(data=df_p[(df_p.slide_scene==s_slide) & (df_pos.loc[:,s_color])],x='DAPI_X',y='DAPI_Y',
                                                                        label=f'{s_color}',s=0.1,color=mpl.cm.tab20.colors[idxs])
        #break
    ax.set_title(f"{s_slide} \n {d_a[s_slide]}", fontsize=16)
    ax.axis('equal')
    ax.set_ylim(ax.get_ylim()[::-1])
    #ax.set_xticklabels('')
    #ax.set_yticklabels('')
    #break
    plt.legend(markerscale=10,framealpha=.5) 
    fig.savefig(f'{codedir}/{s_date}/{s_slide}_leiden{resolution}_scatterplot.png')
    break

In [ ]:
#plot all groups spatially  - manual
df_pos = analyze.celltype_to_bool(df_p,'gated_celltype')
df_pos.columns = [str(item) for item in df_pos.columns]
for s_slide in sorted(set(df_p.slide_scene)):
    s_slide = 'RS-mTMA-5_sceneB07'
    s_slide = 'RS-mTMA-5_sceneA09'
    fig,ax = plt.subplots(figsize=(5,5),dpi=300)
    #plot negative cells
    df_scene = df_p[df_p.index.str.contains(s_slide)]
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=0.1,label=f'')
    #for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
    for idxs, s_color in enumerate(df_pos.columns):
        #print(s_color)
        #s_color = str(s_color_int)
        if len(df_p[(df_p.slide_scene==s_slide) & (df_pos.loc[:,s_color])])>=1:
            #plot positive cells
            ax.scatter(data=df_p[(df_p.slide_scene==s_slide) & (df_pos.loc[:,s_color])],x='DAPI_X',y='DAPI_Y',
                                                                        label=f'{s_color}',s=0.1,color=mpl.cm.tab10.colors[idxs])
        #break
    ax.set_title(f"{s_slide} \n {d_a[s_slide]}", fontsize=16)
    ax.axis('equal')
    if s_slide == 'RS-mTMA-5_sceneB07':
        ax.set_ylim(400,5350)
        ax.set_xlim(400,5350)
    ax.set_ylim(ax.get_ylim()[::-1])
    
    #ax.set_xticklabels('')
    #ax.set_yticklabels('')
    #break
    plt.legend(markerscale=10,framealpha=.1) 
    fig.savefig(f'{codedir}/{s_date}/{s_slide}_gated_cells_scatterplot.png')
    break

## subtype

In [ ]:
s_type = 'leiden' #'gated_celltype' #
df = df_p.loc[:,['CD31',s_type,'slide_scene']].groupby([s_type,'slide_scene']).CD31.count()/df_p.loc[:,['CD31','slide_scene']].groupby(['slide_scene']).CD31.count()
df_prop = df.unstack().T.fillna(0)
ls_col = df_prop.columns.tolist()
adata = sc.AnnData(df_prop)
adata.raw = adata
#reduce dimensionality
sc.tl.pca(adata, svd_solver='auto')
fig,ax=plt.subplots(figsize=(3.5,5))
sc.pl.highest_expr_genes(adata, n_top=48,ax=ax,save=f'Expression_{len(ls_col)}_{s_type}.png')
plt.tight_layout()
sc.pl.pca_variance_ratio(adata,n_pcs=32, log=True)

#umap
for n_neighbors in [5]: #,30
    results_file = f'{s_date}/{s_sample}_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.h5ad'
    print('calc umap')
    # calculate neighbors 
    sc.pp.neighbors(adata, n_neighbors=n_neighbors) #, method='rapids'
    sc.tl.umap(adata)

    #color by markers
    
    figname = f"Umap_markers_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
    title=figname.split('.png')[0].replace('_',' ')
    sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p99.5',ncols=4,size=300,save=figname)
    #survival
    adata.obs['slide'] = adata.obs.index.tolist()
    adata.obs['Group'] = adata.obs.slide.map(d_a).fillna('unknown')
    #adata.obs['Group'] = adata.obs.index.map(d_a).fillna('unknown')
            
    figname = f"Umap_Group_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
    title=figname.split('.png')[0].replace('_',' ')
    fig,ax = plt.subplots(figsize=(3,2), dpi=200)
    sc.pl.umap(adata, color='Group',wspace=.25,save=figname,ax=ax,title=title,size=50)
    
   
    #save results
    if not os.path.exists(results_file):
        adata.write(results_file)
    break

In [ ]:
#sns.clustermap(df_prop)
ls_drop = df_prop[df_prop.index.isin(df_a[df_a.Group=='not_tumor'].slide_scene)].index
df_prop_plot = df_prop.drop(ls_drop)


d_color = dict(zip(['stroma_rich','stroma_poor'],sns.color_palette()[0:2]))
row_colors = df_prop_plot.index.map(d_a).map(d_color)


g = sns.clustermap(df_prop_plot,figsize=(6,6),cmap='viridis',row_colors=row_colors,
        )
#g.fig.suptitle(f'leiden {resolution}',x=.9) 
for label in d_color.keys():
    g.ax_row_dendrogram.bar(0, 0, color=d_color[label],
                            label=label, linewidth=0)
g.ax_row_dendrogram.legend(loc="right", ncol=1)
g.savefig(f'{codedir}/{s_date}/{s_sample}_{s_type}_clustermap_Group_leiden{resolution}.png',dpi=200)
marker_genes = df_prop.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_prop.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()


In [ ]:
df_prop.loc[:,df_p.loc[df_p.celltype=='epithelial','leiden'].unique()]
df_p.loc[df_p.celltype=='epithelial','leiden'].unique()

In [ ]:
df_prop_plot[df_prop_plot.sum(axis=1)>0]

In [ ]:
for s_cell in ['stromal','epithelial','immune']: #'immune'
    ls_index = df_prop[~df_prop.index.isin(df_a[df_a.Group=='not_tumor'].slide_scene)].index
    ls_col = df_p.loc[df_p.gated_celltype==s_cell,'leiden'].unique()
    df_prop_plot = df_prop.loc[ls_index,ls_col]
    df_prop_plot = df_prop_plot[df_prop_plot.sum(axis=1)>0]
    d_color = dict(zip(['stroma_rich','stroma_poor'],sns.color_palette()[0:2]))
    row_colors = df_prop_plot.index.map(d_a).map(d_color)

    g = sns.clustermap(df_prop_plot,figsize=(6,6),cmap='viridis',row_colors=row_colors,z_score=0)
            #vmin=-2,vmax=2)
    g.fig.suptitle(f'{s_cell}',x=.7) 
    for label in d_color.keys():
        g.ax_row_dendrogram.bar(0, 0, color=d_color[label],
                                label=label, linewidth=0)
    g.ax_row_dendrogram.legend(loc="right", ncol=1)
    g.savefig(f'{codedir}/{s_date}/{s_sample}_{s_type}_clustermap_Group_leiden{resolution}_{s_cell}.png',dpi=200)
    marker_genes = df_prop.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
    categories_order = df_prop.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

In [ ]:
#leiden
for resolution in [0.1]: #0.3,
    results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
    sc.tl.leiden(adata,resolution=resolution)
    adata.write(results_file)
    fig,ax = plt.subplots(figsize=(3,3),dpi=200)
    figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}.png'
    sc.pl.umap(adata, color='leiden',ax=ax,title=figname.split('.png')[0].replace('_',' '),wspace=.25,save=figname,size=40)
    break

In [ ]:
#clustermap
df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
df_p['leiden'] = adata.obs['leiden']
d_replace = {}
df_plot = df_p.groupby('leiden').mean()
df_plot.index.name = f'leiden {resolution}'
g = sns.clustermap(df_plot,z_score=1,figsize=(3,4),cmap='viridis',
                   vmin=-1.5,vmax=1.5)
#g.fig.suptitle(f'leiden {resolution}',x=.9) 
g.savefig(f'{codedir}/{s_date}/{s_sample}_clustermap_{s_type}_subtype_{resolution}.png',dpi=200)
marker_genes = df_p.groupby('leiden').mean().iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_p.groupby('leiden').mean().iloc[g.dendrogram_row.reordered_ind,:].index.tolist()